## This will be to analyse the EPS data scraped by DataAssembly/EPS/parse_10q_filings.py
### first analysis is to see how many blanks we have (can I do the rest by hand or not)
### second will be to look for outliers - these will be to look for instances where the LLM screwed up. 


In [24]:
# Analysis for news.db (e.g how many articles are there for each ticker)
import sys, os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, "../..")))
import config
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv("../../"+config.EPS_DATA_CSV)
type(df)

pandas.core.frame.DataFrame

In [26]:
def find_null_eps_10q_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Find rows where all EPS columns are null and the form type is '10-Q'.
    Returns a DataFrame with index, Ticker, and AccessionNumber.
    """
    initial_count = (df['Form Type'] == '10-Q (Quarterly report)').sum()


    mask = (
        (df['Form Type'] == '10-Q (Quarterly report)') &
        (df['quarterly_raw_eps'].isnull()) &
        (df['quarterly_diluted_eps'].isnull()) &
        (df['annual_raw_eps'].isnull()) &
        (df['annual_diluted_eps'].isnull())
    )

    result_df = df.loc[mask, ['Ticker', 'Accession Number']].copy()
    print(f"Total of {initial_count} rows.")
    print(f"[INFO] Found {len(result_df)} rows with all EPS values missing for 10-Q filings. ({len(result_df)*100/initial_count}%)")
    return result_df


In [27]:
missing_df = find_null_eps_10q_rows(df=df)
missing_df.head(10)

Total of 6163 rows.
[INFO] Found 140 rows with all EPS values missing for 10-Q filings. (2.271620963816323%)


,Ticker,Accession Number
178,TER,0001193125-17-255654
260,WDC,0000106040-24-000040
261,WDC,0000106040-24-000022
262,WDC,0000106040-24-000014
263,WDC,0000106040-23-000034
264,WDC,0000106040-23-000017
265,WDC,0000106040-23-000010
269,WDC,0000106040-21-000053
506,ROST,0001206774-12-002608
799,TSLA,0001564590-18-019254


In [28]:
ticker_missing_counts_df = missing_df['Ticker'].value_counts().reset_index()
ticker_missing_counts_df.columns = ['Ticker', 'MissingCount']
print(len(ticker_missing_counts_df))
print(ticker_missing_counts_df)

19
   Ticker  MissingCount
0      BX            23
1     TEL            22
2     KKR            20
3    TSLA            16
4      MS            11
5     WDC             7
6    MCHP             7
7     BAC             7
8     HAS             6
9    ULTA             4
10    AMD             4
11     EG             3
12    PGR             2
13   BKNG             2
14    AJG             2
15   ERIE             1
16   ROST             1
17    PTC             1
18    TER             1
